In [ ]:
!pip install PyMuPDF

In [1]:
import spacy
import pickle
import random
import sys, fitz
import csv

In [ ]:
train_data = pickle.load(open('train_data.pkl', 'rb'))
train_data[0]

In [ ]:
nlp = spacy.blank('en')
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(30):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.3,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)
train_model(train_data)


In [5]:
nlp.to_disk('nlp_model')

In [6]:
nlp_model = spacy.load('nlp_model')

In [ ]:
train_data[0][0]

In [ ]:
doc = nlp_model(train_data[10][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

In [ ]:
import sys, fitz
fname = '/content/test1.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())
    tx = " ".join(text.split('\n'))

doc = nlp_model(tx)
label=[]
text = []
for ent in doc.ents:
    label.append(ent.label_.upper())
    text.append(ent.text)
    print(f'{ent.label_.upper():{30}}- {ent.text}')

In [11]:
rows=[]	
rows.append(text)
	
filename = "output.csv"
	
with open(filename, 'w') as csvfile:

	csvwriter = csv.writer(csvfile)
		
	csvwriter.writerow(label)

	csvwriter.writerows(rows)